### Create centroids from polygons for graduated symbol mapping NEED MORE WORK - STOPPED HERE 03/9/2022
To do this we'll need to project to polygon back to a projected CRS

In [ ]:
# import all the libraries
import numpy as np
import matplotlib as mp
import fiona
import shapely
from shapely.geometry import Polygon, LineString, Point
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
import matplotlib.pyplot as plt
from rasterio.plot import show
import mapclassify
from functools import reduce

#change default figure size
plt.rcParams['figure.figsize'] = (12,12)

# show all columns
pd.set_option("display.max_columns", None)

In [ ]:
# set a variable with parameters for USA Contiguous Equidistant
project_params  = '+proj=eqdc +lat_0=39 +lon_0=-96 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs'

# reproject and assign to new variables
cdps_svis_whp_ec = cdps_svis_whp.to_crs(project_params)

In [ ]:
# convert data to centroid and results in a geoseries
cdps_svis_whp_ctr = cdps_svis_whp_ec['center']=cdps_svis_whp.centroid

In [ ]:
cdps_svis_whp_ctr_df = gpd.GeoDataFrame(geometry=gpd.GeoSeries(cdps_svis_whp_ctr))

In [ ]:
cdps_svis_whp_ctr_df.info()

In [ ]:
# spatially join centroids to polygons to get the data for graduated symbol mapping
cdps_svis_whp_pts = cdps_svis_whp_ctr_df.sjoin(cdps_svis_whp_ec, how='right', predicate='intersects')

In [ ]:
cdps_svis_whp_pts.info()

In [ ]:
cdps_svis_whp_pts.head(10)

In [ ]:
# plot the resulting dataframe interactively
cdps_svis_whp_pts.explore(tooltip=['GEOID','NAMELSAD_x', 'whp_score', 'se_wm','hcd_wm','m_wm','htt_wm','overall_wm'])